In [4]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=bd2cd88c59b2dbc764c7772ed593c2eb8e4e6e5145787adef2018792fdd9eac6
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [5]:
# Import findspark and initialize.
import findspark
findspark.init()

In [6]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [9]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show the delayed flight data.
home_sales_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [12]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')


In [39]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
    SELECT EXTRACT(YEAR FROM date) AS Year, ROUND(AVG(price), 2) AS average_price FROM home_sales
    WHERE bedrooms == 4
    GROUP BY year
    ORDER BY year
""").show()


+----+-------------+
|Year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [40]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
    SELECT date_built AS year, ROUND(AVG(price), 2) AS average_price FROM home_sales
    WHERE bedrooms == 3 AND bathrooms == 3
    GROUP BY year
    ORDER BY year
""").show()


+----+-------------+
|year|average_price|
+----+-------------+
|2010|    292859.62|
|2011|    291117.47|
|2012|    293683.19|
|2013|    295962.27|
|2014|    290852.27|
|2015|     288770.3|
|2016|    290555.07|
|2017|    292676.79|
+----+-------------+



In [42]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
    SELECT date_built AS year, ROUND(AVG(price), 2) AS average_price FROM home_sales
    WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000
    GROUP BY year
    ORDER BY year
""").show()


+----+-------------+
|year|average_price|
+----+-------------+
|2010|    285010.22|
|2011|    276553.81|
|2012|    307539.97|
|2013|    303676.79|
|2014|    298264.72|
|2015|    297609.97|
|2016|     293965.1|
|2017|    280317.58|
+----+-------------+



In [45]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""SELECT view, ROUND(AVG(price), 2) AS average_price_home FROM home_sales
            WHERE price >= 350000
            GROUP BY view
            ORDER BY average_price_home
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|average_price_home|
+----+------------------+
|  33|         393973.32|
|  37|         394964.36|
|   6|         395655.38|
|  42|          396964.5|
|   2|         397389.25|
|  29|         397771.65|
|  30|          397862.0|
|  38|         398418.37|
|  47|          398447.5|
|  17|         398474.49|
|  14|         398570.03|
|   8|         398592.71|
|   3|          398867.6|
|  13|         398917.98|
|  19|         398953.17|
|  39|         399234.22|
|  18|         399332.91|
|  32|         399367.72|
|  20|         399522.81|
|  27|         399537.66|
+----+------------------+
only showing top 20 rows

--- 0.7099795341491699 seconds ---


In [46]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [47]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [50]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price_home
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY average_price_home
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+------------------+
|view|average_price_home|
+----+------------------+
|  33|         393973.32|
|  37|         394964.36|
|   6|         395655.38|
|  42|          396964.5|
|   2|         397389.25|
|  29|         397771.65|
|  30|          397862.0|
|  38|         398418.37|
|  47|          398447.5|
|  17|         398474.49|
|  14|         398570.03|
|   8|         398592.71|
|   3|          398867.6|
|  13|         398917.98|
|  19|         398953.17|
|  39|         399234.22|
|  18|         399332.91|
|  32|         399367.72|
|  20|         399522.81|
|  27|         399537.66|
+----+------------------+
only showing top 20 rows

--- 0.5203146934509277 seconds ---


In [52]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_data")

In [54]:
# 11. Read the formatted parquet data.
df_p=spark.read.parquet('home_sales_data')

In [56]:
# 12. Create a temporary table for the parquet data.
df_p.createOrReplaceTempView("temp_table_parquet")
df_p.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [58]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price_home
    FROM temp_table_parquet
    WHERE price >= 350000
    GROUP BY view
    ORDER BY average_price_home
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|average_price_home|
+----+------------------+
|  33|         393973.32|
|  37|         394964.36|
|   6|         395655.38|
|  42|          396964.5|
|   2|         397389.25|
|  29|         397771.65|
|  30|          397862.0|
|  38|         398418.37|
|  47|          398447.5|
|  17|         398474.49|
|  14|         398570.03|
|   8|         398592.71|
|   3|          398867.6|
|  13|         398917.98|
|  19|         398953.17|
|  39|         399234.22|
|  18|         399332.91|
|  32|         399367.72|
|  20|         399522.81|
|  27|         399537.66|
+----+------------------+
only showing top 20 rows

--- 0.5184652805328369 seconds ---


In [60]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [61]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False